<a href="https://colab.research.google.com/github/dkshjn/Technoculture/blob/main/DSPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# @title Initial Setup
%load_ext autoreload
%autoreload 2

import sys
import os

try: # When on google Colab, let's clone the notebook so we download the cache.
    import google.colab
    repo_path = 'dspy'
    !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_path
except:
    repo_path = '.'

if repo_path not in sys.path:
    sys.path.append(repo_path)

# Set up the cache for this notebook
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(repo_path, 'cache')

import pkg_resources # Install the package if it's not installed
if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install -U pip
    !pip install dspy-ai
    # !pip install -e $repo_path

import dspy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Already up to date.


Using turbo as LM and ColbertV2 as retrieval model. Wikipedia 2017 used.

Error occuring due to version clash.
dspy requires openai (0.28.1) but OpenAI is available in the latest version openai-1.6.1.
That's why import error occurs when run.
ImportError: cannot import name 'OpenAI' from 'openai'

In [10]:
!pip install openai
from openai import OpenAI
from os import getenv

# gets API Key from environment variable OPENAI_API_KEY
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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",
)

completion = client.chat.completions.create(
  # extra_headers={
  #   "HTTP-Referer": $YOUR_SITE_URL, # Optional, for including your app on openrouter.ai rankings.
  #   "X-Title": $YOUR_APP_NAME, # Optional. Shows in rankings on openrouter.ai.
  # },
  model="openchat/openchat-7b",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of life?",
    },
  ],
)
print(completion.choices[0].message.content)

ImportError: ignored

In [ ]:
turbo = dspy.OpenAI(api_base="https://openrouter.ai/api/v1", api_key="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",model='openchat/openchat-7b')
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

dspy.settings.configure(lm=turbo, rm=colbertv2_wiki17_abstracts)

In [9]:
from dspy.datasets import HotPotQA

dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]
len(trainset), len(devset)

Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]

(20, 50)

Two datasets Train and Dev.

In [4]:
attributes_and_methods = dir(dataset)
print(attributes_and_methods)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_dev', '_dev_', '_shuffle_and_sample', '_test', '_train', '_train_', 'dev', 'dev_seed', 'dev_size', 'do_shuffle', 'name', 'prepare_by_seed', 'reset_seeds', 'test', 'test_seed', 'test_size', 'train', 'train_seed', 'train_size']


In [5]:
train_example1 = trainset[0]
print(f"Question1: {train_example1.question}")
print(f"Answer1: {train_example1.answer}\n")

train_example2 = trainset[1]
print(f"Question2: {train_example2.question}")
print(f"Answer2: {train_example2.answer}\n")

train_example3 = trainset[2]
print(f"Question3: {train_example3.question}")
print(f"Answer3: {train_example3.answer}\n")


Question1: At My Window was released by which American singer-songwriter?
Answer1: John Townes Van Zandt

Question2: which  American actor was Candace Kita  guest starred with 
Answer2: Bill Murray

Question3: Which of these publications was most recently published, Who Put the Bomp or Self?
Answer3: Self



In [6]:
dev_example1 = devset[15]
print(f"Question: {dev_example1.question}")
print(f"Answer: {dev_example1.answer}")
print(f"Relevant Wikipedia Titles: {dev_example1.gold_titles}\n")

dev_example2 = devset[16]
print(f"Question: {dev_example2.question}")
print(f"Answer: {dev_example2.answer}")
print(f"Relevant Wikipedia Titles: {dev_example2.gold_titles}\n")

dev_example3 = devset[18]
print(f"Question: {dev_example3.question}")
print(f"Answer: {dev_example3.answer}")
print(f"Relevant Wikipedia Titles: {dev_example3.gold_titles}\n")

Question: What is the name of this region of Italy, referring to the medieval March of Ancona and nearby marches of Camerino and Fermo, where the comune Pollenza is located?
Answer: Marche
Relevant Wikipedia Titles: {'Pollenza', 'Marche'}

Question: William Hughes Miller was born in a city with how many inhabitants ?
Answer: 7,402 at the 2010 census
Relevant Wikipedia Titles: {'William Hughes Miller', 'Kosciusko, Mississippi'}

Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?
Answer: English
Relevant Wikipedia Titles: {'Robert Irvine', 'Restaurant: Impossible'}



In [7]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [ ]:
generate_answer = dspy.Predict(BasicQA)

pred = generate_answer(question=dev_example3.question)


print(f"Question: {dev_example3.question}")
print(f"Predicted Answer: {pred.answer}")

In [ ]:
# Define the predictor. Notice we're just changing the class. The signature BasicQA is unchanged.
generate_answer_with_chain_of_thought = dspy.ChainOfThought(BasicQA)

# Call the predictor on the same input.
pred = generate_answer_with_chain_of_thought(question=dev_example3.question)

# Print the input, the chain of thought, and the prediction.
print(f"Question: {dev_example3.question}")
print(f"Thought: {pred.rationale.split('.', 1)[1].strip()}")
print(f"Predicted Answer: {pred.answer}")

In [ ]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [ ]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")

    def forward(self, question):
        context = self.retrieve(question).passages
        answer = self.generate_answer(context=context, question=question)
        return answer

Teleprompter

In [ ]:
from dspy.teleprompt import BootstrapFewShot

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

# Compile!
compiled_rag = teleprompter.compile(RAG(), trainset=trainset)



In [ ]:
# Ask any question you like to this simple RAG program.
my_question = "What castle did David Gregory inherit?"

# Get the prediction. This contains `pred.context` and `pred.answer`.
pred = compiled_rag(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

In [ ]:
generate_answer_with_chain_of_thought = dspy.ChainOfThought(BasicQA)

pred = generate_answer_with_chain_of_thought(question=dev_example3.question)


print(f"Question: {dev_example3.question}")
print(f"Thought: {pred.rationale.split('.', 1)[1].strip()}")
print(f"Predicted Answer: {pred.answer}")